# Money App

This notebook is to design and develop all-encompassing money management system for Oscar and Rose.


## Components
The components are the steps of the money pipeline. These operations on these steps are the core logic of the system and should not be modified without a system design review.

### Income
Income refers to any source of money coming to us. The most obvious instance of income is our salaries.

### Bills
Bills are the payments performed in cycles. These include mortgage, utility bills and similar.

### Expenses
Expenses are any ad-hoc payments for goods or services. These include food, cleaning and shopping.

### Savings
This is the share of money left after using our income to pay for bills and expenses that is saved.

### Investments
This is the share of money left after savings that is put back in the market to generate a return.

## Money Pipeline
The money processing pipeline is as shown below.

<!-- ![pipeline](money-pipeline.png) -->

## Input Data
- Savings accumulated
- Income from salaries or investments
- Bills for the past month
- Expenses for the past month
- Investments in the past month

## Parameters
Parameters are the parts of the system we can modify to make it work for our financial situation and expectations. These include:
- Saving pots and shares for each of them
- How bills and expenses are split between us
- The savings accounts we use and what pots go each one holds

In [3]:
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta

from data.D2312 import INPUTS, PARAMS, DATE


def inspect_inputs_dict():
    pass


def inspect_params_dict():
    pass


def get_income(inputs):
    income_oscar = sum(source["oscar"] for source in inputs["income"].values())
    income_rose = sum(source["rose"] for source in inputs["income"].values())
    return income_oscar, income_rose


def get_bills(inputs):
    bills_oscar = sum(source["oscar"] for source in inputs["bills"].values())
    bills_rose = sum(source["rose"] for source in inputs["bills"].values())
    return bills_oscar, bills_rose


def get_expenses(inputs):
    expenses_oscar = sum(source["oscar"] for source in inputs["expenses"].values())
    expenses_rose = sum(source["rose"] for source in inputs["expenses"].values())
    return expenses_oscar, expenses_rose


def get_investments(inputs):
    investments_oscar = sum(
        source["oscar"] for source in inputs["investments"].values()
    )
    investments_rose = sum(source["rose"] for source in inputs["investments"].values())
    return investments_oscar, investments_rose


def get_spending_total(inputs):
    return sum(get_bills(inputs)) + sum(get_expenses(inputs))


def get_spending_share(inputs, params):
    if params["split"] == "proportional_to_income":
        income_oscar, income_rose = get_income(inputs)
        split_percentage_oscar = income_oscar / (income_oscar + income_rose)
        split_percentage_rose = income_rose / (income_oscar + income_rose)
        return split_percentage_oscar, split_percentage_rose
    else:
        raise ValueError("split mode not supported")


def get_expected_spending(inputs, params):
    spending_share_oscar, spending_share_rose = get_spending_share(inputs, params)
    expected_spending_oscar = get_spending_total(inputs) * spending_share_oscar
    expected_spending_rose = get_spending_total(inputs) * spending_share_rose
    return expected_spending_oscar, expected_spending_rose


def get_actual_spending(inputs):
    actual_spending_oscar = (
        sum(source["oscar"] for source in inputs["bills"].values())
        + sum(source["oscar"] for source in inputs["expenses"].values())
        + sum(source["oscar"] for source in inputs["investments"].values())
    )
    actual_spending_rose = (
        sum(source["rose"] for source in inputs["bills"].values())
        + sum(source["rose"] for source in inputs["expenses"].values())
        + sum(source["rose"] for source in inputs["investments"].values())
    )
    return actual_spending_oscar, actual_spending_rose


def get_split_difference(inputs, params):
    expected_spending_oscar, expected_spending_rose = get_expected_spending(
        inputs, params
    )
    actual_spending_oscar, actual_spending_rose = get_actual_spending(inputs)
    split_difference_oscar = expected_spending_oscar - actual_spending_oscar
    split_difference_rose = expected_spending_rose - actual_spending_rose
    return split_difference_oscar, split_difference_rose


def get_savings_interest(inputs):
    return sum(source for source in inputs["savings_interest"].values())


def get_personal_spending(inputs):
    return sum(source for source in inputs["personal_spending"].values())


def get_savings(inputs):
    income_oscar, income_rose = get_income(inputs)
    actual_spending_oscar, actual_spending_rose = get_actual_spending(inputs)
    savings_oscar = income_oscar - actual_spending_oscar
    savings_rose = income_rose - actual_spending_rose
    return savings_oscar, savings_rose


def get_saving_pots(params):
    return list(params["saving_pots_shares"].keys())


def get_saving_pots_contribution(inputs, params):
    savings_total = (
        sum(get_savings(inputs))
        + get_savings_interest(inputs)
        - get_personal_spending(inputs)
    )
    saving_pots_contributions = {}
    for pot in get_saving_pots(params):
        saving_pots_contributions[pot] = (
            savings_total * params["saving_pots_shares"][pot]
        )
    return saving_pots_contributions


def get_saving_pots_contribution_per_account(inputs, params):
    saving_pots_contribution = get_saving_pots_contribution(inputs, params)
    return {
        "easy_access_account": sum(
            [
                saving_pots_contribution[pot]
                for pot in saving_pots_contribution
                if pot in params["easy_access_pots"]
            ]
        ),
        "high_yield_account": sum(
            [
                saving_pots_contribution[pot]
                for pot in saving_pots_contribution
                if pot in params["high_yield_pots"]
            ]
        ),
    }


def get_saving_pots_projection_df(inputs, params, date, months=12):
    saving_pots_projection_df = pd.DataFrame(
        columns=["month"] + get_saving_pots(params)
    )
    saving_pots_projection_df.loc[0] = {
        **{"month": datetime.strptime(date, "%Y-%m-%d").strftime("%b %y")},
        **inputs["current_savings"],
    }
    start_date = datetime.strptime(date, "%Y-%m-%d")
    saving_pots_contribution = get_saving_pots_contribution(inputs, params)
    for i in range(1, months + 1):
        saving_pots_projection_df.loc[i] = {
            **{
                "month": (start_date + relativedelta(months=i)).strftime("%b %y"),
            },
            **{
                pot: (
                    saving_pots_projection_df.loc[i - 1][pot]
                    + saving_pots_contribution[pot]
                )
                for pot in get_saving_pots(params)
            },
        }
    return saving_pots_projection_df

print(get_saving_pots_contribution(INPUTS, PARAMS))
print(get_saving_pots_contribution_per_account(INPUTS, PARAMS))
get_saving_pots_projection_df(INPUTS, PARAMS, DATE, months=12)

{'new_house': 1330.356, 'car': 886.9040000000001, 'current_house': 665.178, 'holidays': 665.178, 'wedding': 443.45200000000006, 'children': 221.72600000000003, 'family': 221.72600000000003}
{'easy_access_account': 1995.534, 'high_yield_account': 2438.9860000000003}


,month,new_house,car,current_house,holidays,wedding,children,family
0,Dec 23,6720.000,15120.000,11760.000,0.000,0.000,0.000,0.000
1,Jan 24,8050.356,16006.904,12425.178,665.178,443.452,221.726,221.726
2,Feb 24,9380.712,16893.808,13090.356,1330.356,886.904,443.452,443.452
3,Mar 24,10711.068,17780.712,13755.534,1995.534,1330.356,665.178,665.178
4,Apr 24,12041.424,18667.616,14420.712,2660.712,1773.808,886.904,886.904
5,May 24,13371.780,19554.520,15085.890,3325.890,2217.260,1108.630,1108.630
6,Jun 24,14702.136,20441.424,15751.068,3991.068,2660.712,1330.356,1330.356
7,Jul 24,16032.492,21328.328,16416.246,4656.246,3104.164,1552.082,1552.082
8,Aug 24,17362.848,22215.232,17081.424,5321.424,3547.616,1773.808,1773.808
9,Sep 24,18693.204,23102.136,17746.602,5986.602,3991.068,1995.534,1995.534
